### Synthetic Data Backtest

Most of the functions below can be found under:

* Tool/backtest 
* Sample_data/make_data

A quick slideshow: [Seven Sins of Backtesting](https://newyork.qwafafew.org/wp-content/uploads/sites/4/2015/10/Luo_20150128.pdf)

The below is a code snippet for trading rules optimization. As for how to use it, I leave it to the reader.

The key for this exercise was to use real-HFT data to deduce a certain outcome.. which I don't have.

Hence, I will just replace this exercise with multiprocessing methods (Parallelization!).

If you are keen on generating synthetic data for your research, copy the code snippets [Generate synthetic raw data](https://gist.github.com/boyboi86/5e00faf48f60abfdbe838fbdee269471) in my gist.

Contact: boyboi86@gmail.com

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import research as rs
from random import gauss
from itertools import product

%matplotlib inline

Num of CPU core:  4
Machine info:  Windows-10-10.0.18362-SP0
Python 3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]
Numpy 1.18.5
Pandas 1.0.4


C:\Users\Wei_X\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


<Figure size 1500x800 with 1 Axes>

In [2]:
dict0 = {'a':['1', '2'], 'b':['+', '*'], 'c':['!','A']}
for a in dict0['a']:
    for b in dict0['b']:
        for c in dict0['c']:
            print({'a':a, 'b':b, 'c':c})

{'a': '1', 'b': '+', 'c': '!'}
{'a': '1', 'b': '+', 'c': 'A'}
{'a': '1', 'b': '*', 'c': '!'}
{'a': '1', 'b': '*', 'c': 'A'}
{'a': '2', 'b': '+', 'c': '!'}
{'a': '2', 'b': '+', 'c': 'A'}
{'a': '2', 'b': '*', 'c': '!'}
{'a': '2', 'b': '*', 'c': 'A'}


**Note**

The above is scalar operation. Notice loop after loop after loop.

The below is vectorization operation, which is what we will do for Monte-Carlos.

The result is actually the same.

In [3]:
from itertools import product
jobs = (dict(zip(dict0, i )) for i in product(*dict0.values()))
for i in jobs: print(i)

{'a': '1', 'b': '+', 'c': '!'}
{'a': '1', 'b': '+', 'c': 'A'}
{'a': '1', 'b': '*', 'c': '!'}
{'a': '1', 'b': '*', 'c': 'A'}
{'a': '2', 'b': '+', 'c': '!'}
{'a': '2', 'b': '+', 'c': 'A'}
{'a': '2', 'b': '*', 'c': '!'}
{'a': '2', 'b': '*', 'c': 'A'}


**Note**

Given the parameters below. It should look like the below.

In [4]:
forecast = [10,5,0,-5,-10]
half_life = [5,10,25,50,100]
loss_range = profit_range = np.linspace(.5,10,20)
sigma = [1]

coeff = dict({'forecast': forecast,
              'half_life': half_life,
              'sigma':sigma,
              'profit_range': profit_range,
              'loss_range':loss_range})
jobs = list(dict(zip(coeff, idx)) for idx in product(*coeff.values()))

print(len(jobs))

10000


**Note**

The below is the end product for a parallized Monte-Carlos Simulation, for optimizied Trading Rule Algorithm.

Primarily we will use it to create scenarios for our models to be backtest with.

In [5]:
def _opt_tr(coeff: dict = None,
            n_run: float = 100,
            max_period: int = 100,
            seed: int = 0):
    
    phi, output1 = 2 ** (-1/ coeff['half_life']), []
    for iter_ in range(int(n_run)):
        p, hold_period, count = seed, 0, 0
        while True:
            p = (1 - phi) * coeff['forecast'] + phi * p + coeff['sigma'] * gauss(0,1)
            cp = p - seed
            hold_period += 1
            if coeff['profit_range'] < cp or -coeff['loss_range'] > cp or hold_period > max_period:
                output1.append(cp)
                break
    mean, std = np.mean(output1), np.std(output1)
    return {'profit': coeff['profit_range'], 'loss': coeff['loss_range'], 'mean': mean, 'std': std, 'sharpe': mean/std}

def opt_tr(profit_range: list = np.linspace(.5, 10, 20),
          loss_range: list = np.linspace(.5, 10, 20),
          sigma: list = [1],
          n_run: float = 100,
          max_period: int = 100,
          forecast: list = [10,5,0,-5,-10], 
          half_life: list = [5,10,25,50,100],
          num_threads: int = 1):
    
    if profit_range is None:
        profit_range = np.linspace(0,10,21)
        
    if loss_range is None:
        loss_range = np.linspace(0,10,21)
        
    _coeff_list = []
    _coeff = dict({'forecast': forecast,
                   'half_life': half_life,
                   'sigma':sigma,
                   'profit_range': profit_range,
                   'loss_range':loss_range})
    
    _coeff = list(dict(zip(_coeff, idx )) for idx in product(*_coeff.values()))
    out = []
    for coeff in _coeff:
        jobs = [{'func': _opt_tr,
                   'coeff': coeff,
                   'n_run': n_run, 
                   'max_period': max_period}]
        
        if num_threads == 1:
            out_ = rs.process_jobs_(jobs)
            out.append(out_)
        else:
            out_ = rs.process_jobs(jobs, num_threads = num_threads)
            out.append(out_)

    return out

**Note**

After parallization. Total runs for estimated 1,000,000 loops took only less than 1 min

* 2020-06-14 17:02:17.058014 (Beginning!)

* 2020-06-14 17:03:06.231600 (End!)

**Note**

Disclaimer! If I run more than 1 thread, it becomes incredibly slow. 

So take note, if single process is fast enough then try not to increase the process required. 

The same thing happened to me when I used sklearn.

Don't be too greedy.

In [6]:
df = opt_tr()

2020-06-14 17:02:17.058014 done after 0.00 mins
2020-06-14 17:02:17.059013 done after 0.00 mins
2020-06-14 17:02:17.059013 done after 0.00 mins
2020-06-14 17:02:17.060009 done after 0.00 mins
2020-06-14 17:02:17.061010 done after 0.00 mins
2020-06-14 17:02:17.061010 done after 0.00 mins
2020-06-14 17:02:17.063014 done after 0.00 mins
2020-06-14 17:02:17.066005 done after 0.00 mins
2020-06-14 17:02:17.066005 done after 0.00 mins
2020-06-14 17:02:17.067003 done after 0.00 mins
2020-06-14 17:02:17.067990 done after 0.00 mins
2020-06-14 17:02:17.068990 done after 0.00 mins
2020-06-14 17:02:17.068990 done after 0.00 mins
2020-06-14 17:02:17.069984 done after 0.00 mins
2020-06-14 17:02:17.069984 done after 0.00 mins
2020-06-14 17:02:17.069984 done after 0.00 mins
2020-06-14 17:02:17.070980 done after 0.00 mins
2020-06-14 17:02:17.071978 done after 0.00 mins
2020-06-14 17:02:17.071978 done after 0.00 mins
2020-06-14 17:02:17.071978 done after 0.00 mins
2020-06-14 17:02:17.072975 done after 0.

2020-06-14 17:02:17.252637 done after 0.00 mins
2020-06-14 17:02:17.252637 done after 0.00 mins
2020-06-14 17:02:17.252637 done after 0.00 mins
2020-06-14 17:02:17.252637 done after 0.00 mins
2020-06-14 17:02:17.252637 done after 0.00 mins
2020-06-14 17:02:17.252637 done after 0.00 mins
2020-06-14 17:02:17.252637 done after 0.00 mins
2020-06-14 17:02:17.252637 done after 0.00 mins
2020-06-14 17:02:17.252637 done after 0.00 mins
2020-06-14 17:02:17.252637 done after 0.00 mins
2020-06-14 17:02:17.252637 done after 0.00 mins
2020-06-14 17:02:17.252637 done after 0.00 mins
2020-06-14 17:02:17.252637 done after 0.00 mins
2020-06-14 17:02:17.268257 done after 0.00 mins
2020-06-14 17:02:17.268257 done after 0.00 mins
2020-06-14 17:02:17.268257 done after 0.00 mins
2020-06-14 17:02:17.268257 done after 0.00 mins
2020-06-14 17:02:17.268257 done after 0.00 mins
2020-06-14 17:02:17.268257 done after 0.00 mins
2020-06-14 17:02:17.268257 done after 0.00 mins
2020-06-14 17:02:17.268257 done after 0.

2020-06-14 17:02:17.725568 done after 0.00 mins
2020-06-14 17:02:17.725568 done after 0.00 mins
2020-06-14 17:02:17.725568 done after 0.00 mins
2020-06-14 17:02:17.725568 done after 0.00 mins
2020-06-14 17:02:17.725568 done after 0.00 mins
2020-06-14 17:02:17.725568 done after 0.00 mins
2020-06-14 17:02:17.725568 done after 0.00 mins
2020-06-14 17:02:17.725568 done after 0.00 mins
2020-06-14 17:02:17.725568 done after 0.00 mins
2020-06-14 17:02:17.725568 done after 0.00 mins
2020-06-14 17:02:17.725568 done after 0.00 mins
2020-06-14 17:02:17.725568 done after 0.00 mins
2020-06-14 17:02:17.741190 done after 0.00 mins
2020-06-14 17:02:17.741190 done after 0.00 mins
2020-06-14 17:02:17.741190 done after 0.00 mins
2020-06-14 17:02:17.741190 done after 0.00 mins
2020-06-14 17:02:17.741190 done after 0.00 mins
2020-06-14 17:02:17.741190 done after 0.00 mins
2020-06-14 17:02:17.741190 done after 0.00 mins
2020-06-14 17:02:17.741190 done after 0.00 mins
2020-06-14 17:02:17.741190 done after 0.

2020-06-14 17:02:18.312661 done after 0.00 mins
2020-06-14 17:02:18.326580 done after 0.00 mins
2020-06-14 17:02:18.335557 done after 0.00 mins
2020-06-14 17:02:18.340542 done after 0.00 mins
2020-06-14 17:02:18.344531 done after 0.00 mins
2020-06-14 17:02:18.348520 done after 0.00 mins
2020-06-14 17:02:18.353507 done after 0.00 mins
2020-06-14 17:02:18.358493 done after 0.00 mins
2020-06-14 17:02:18.363481 done after 0.00 mins
2020-06-14 17:02:18.370462 done after 0.00 mins
2020-06-14 17:02:18.373453 done after 0.00 mins
2020-06-14 17:02:18.377442 done after 0.00 mins
2020-06-14 17:02:18.379545 done after 0.00 mins
2020-06-14 17:02:18.379545 done after 0.00 mins
2020-06-14 17:02:18.379545 done after 0.00 mins
2020-06-14 17:02:18.395209 done after 0.00 mins
2020-06-14 17:02:18.395209 done after 0.00 mins
2020-06-14 17:02:18.395209 done after 0.00 mins
2020-06-14 17:02:18.395209 done after 0.00 mins
2020-06-14 17:02:18.395209 done after 0.00 mins
2020-06-14 17:02:18.410788 done after 0.

2020-06-14 17:02:18.705132 done after 0.00 mins
2020-06-14 17:02:18.705132 done after 0.00 mins
2020-06-14 17:02:18.717321 done after 0.00 mins
2020-06-14 17:02:18.723308 done after 0.00 mins
2020-06-14 17:02:18.728295 done after 0.00 mins
2020-06-14 17:02:18.733279 done after 0.00 mins
2020-06-14 17:02:18.734276 done after 0.00 mins
2020-06-14 17:02:18.736271 done after 0.00 mins
2020-06-14 17:02:18.738265 done after 0.00 mins
2020-06-14 17:02:18.740260 done after 0.00 mins
2020-06-14 17:02:18.742254 done after 0.00 mins
2020-06-14 17:02:18.746246 done after 0.00 mins
2020-06-14 17:02:18.749236 done after 0.00 mins
2020-06-14 17:02:18.752228 done after 0.00 mins
2020-06-14 17:02:18.755220 done after 0.00 mins
2020-06-14 17:02:18.758213 done after 0.00 mins
2020-06-14 17:02:18.761204 done after 0.00 mins
2020-06-14 17:02:18.762072 done after 0.00 mins
2020-06-14 17:02:18.762072 done after 0.00 mins
2020-06-14 17:02:18.762072 done after 0.00 mins
2020-06-14 17:02:18.762072 done after 0.

...

2020-06-14 17:03:01.571942 done after 0.00 mins
2020-06-14 17:03:01.571942 done after 0.00 mins
2020-06-14 17:03:01.571942 done after 0.00 mins
2020-06-14 17:03:01.571942 done after 0.00 mins
2020-06-14 17:03:01.571942 done after 0.00 mins
2020-06-14 17:03:01.571942 done after 0.00 mins
2020-06-14 17:03:01.587563 done after 0.00 mins
2020-06-14 17:03:01.587563 done after 0.00 mins
2020-06-14 17:03:01.587563 done after 0.00 mins
2020-06-14 17:03:01.603184 done after 0.00 mins
2020-06-14 17:03:01.603184 done after 0.00 mins
2020-06-14 17:03:01.603184 done after 0.00 mins
2020-06-14 17:03:01.618807 done after 0.00 mins
2020-06-14 17:03:01.618807 done after 0.00 mins
2020-06-14 17:03:01.634427 done after 0.00 mins
2020-06-14 17:03:01.634427 done after 0.00 mins
2020-06-14 17:03:01.634427 done after 0.00 mins
2020-06-14 17:03:01.634427 done after 0.00 mins
2020-06-14 17:03:01.634427 done after 0.00 mins
2020-06-14 17:03:01.634427 done after 0.00 mins
2020-06-14 17:03:01.634427 done after 0.

2020-06-14 17:03:02.307256 done after 0.00 mins
2020-06-14 17:03:02.314232 done after 0.00 mins
2020-06-14 17:03:02.315230 done after 0.00 mins
2020-06-14 17:03:02.316228 done after 0.00 mins
2020-06-14 17:03:02.318259 done after 0.00 mins
2020-06-14 17:03:02.319261 done after 0.00 mins
2020-06-14 17:03:02.320256 done after 0.00 mins
2020-06-14 17:03:02.321214 done after 0.00 mins
2020-06-14 17:03:02.323242 done after 0.00 mins
2020-06-14 17:03:02.324239 done after 0.00 mins
2020-06-14 17:03:02.326237 done after 0.00 mins
2020-06-14 17:03:02.329194 done after 0.00 mins
2020-06-14 17:03:02.331185 done after 0.00 mins
2020-06-14 17:03:02.333180 done after 0.00 mins
2020-06-14 17:03:02.336172 done after 0.00 mins
2020-06-14 17:03:02.339164 done after 0.00 mins
2020-06-14 17:03:02.342156 done after 0.00 mins
2020-06-14 17:03:02.346186 done after 0.00 mins
2020-06-14 17:03:02.349172 done after 0.00 mins
2020-06-14 17:03:02.353163 done after 0.00 mins
2020-06-14 17:03:02.357161 done after 0.

2020-06-14 17:03:03.092433 done after 0.00 mins
2020-06-14 17:03:03.110420 done after 0.00 mins
2020-06-14 17:03:03.122355 done after 0.00 mins
2020-06-14 17:03:03.130333 done after 0.00 mins
2020-06-14 17:03:03.137312 done after 0.00 mins
2020-06-14 17:03:03.147287 done after 0.00 mins
2020-06-14 17:03:03.157258 done after 0.00 mins
2020-06-14 17:03:03.163332 done after 0.00 mins
2020-06-14 17:03:03.163332 done after 0.00 mins
2020-06-14 17:03:03.163332 done after 0.00 mins
2020-06-14 17:03:03.163332 done after 0.00 mins
2020-06-14 17:03:03.163332 done after 0.00 mins
2020-06-14 17:03:03.178955 done after 0.00 mins
2020-06-14 17:03:03.178955 done after 0.00 mins
2020-06-14 17:03:03.178955 done after 0.00 mins
2020-06-14 17:03:03.178955 done after 0.00 mins
2020-06-14 17:03:03.194575 done after 0.00 mins
2020-06-14 17:03:03.194575 done after 0.00 mins
2020-06-14 17:03:03.194575 done after 0.00 mins
2020-06-14 17:03:03.210197 done after 0.00 mins
2020-06-14 17:03:03.210197 done after 0.

2020-06-14 17:03:04.286587 done after 0.00 mins
2020-06-14 17:03:04.286587 done after 0.00 mins
2020-06-14 17:03:04.286587 done after 0.00 mins
2020-06-14 17:03:04.286587 done after 0.00 mins
2020-06-14 17:03:04.286587 done after 0.00 mins
2020-06-14 17:03:04.286587 done after 0.00 mins
2020-06-14 17:03:04.302208 done after 0.00 mins
2020-06-14 17:03:04.302208 done after 0.00 mins
2020-06-14 17:03:04.302208 done after 0.00 mins
2020-06-14 17:03:04.302208 done after 0.00 mins
2020-06-14 17:03:04.302208 done after 0.00 mins
2020-06-14 17:03:04.302208 done after 0.00 mins
2020-06-14 17:03:04.302208 done after 0.00 mins
2020-06-14 17:03:04.302208 done after 0.00 mins
2020-06-14 17:03:04.317831 done after 0.00 mins
2020-06-14 17:03:04.317831 done after 0.00 mins
2020-06-14 17:03:04.317831 done after 0.00 mins
2020-06-14 17:03:04.333452 done after 0.00 mins
2020-06-14 17:03:04.340315 done after 0.00 mins
2020-06-14 17:03:04.346299 done after 0.00 mins
2020-06-14 17:03:04.359265 done after 0.

2020-06-14 17:03:05.137211 done after 0.00 mins
2020-06-14 17:03:05.152891 done after 0.00 mins
2020-06-14 17:03:05.160872 done after 0.00 mins
2020-06-14 17:03:05.167851 done after 0.00 mins
2020-06-14 17:03:05.175829 done after 0.00 mins
2020-06-14 17:03:05.184805 done after 0.00 mins
2020-06-14 17:03:05.193782 done after 0.00 mins
2020-06-14 17:03:05.198804 done after 0.00 mins
2020-06-14 17:03:05.203786 done after 0.00 mins
2020-06-14 17:03:05.210773 done after 0.00 mins
2020-06-14 17:03:05.212821 done after 0.00 mins
2020-06-14 17:03:05.212821 done after 0.00 mins
2020-06-14 17:03:05.228444 done after 0.00 mins
2020-06-14 17:03:05.228444 done after 0.00 mins
2020-06-14 17:03:05.244102 done after 0.00 mins
2020-06-14 17:03:05.244102 done after 0.00 mins
2020-06-14 17:03:05.270210 done after 0.00 mins
2020-06-14 17:03:05.285172 done after 0.00 mins
2020-06-14 17:03:05.297214 done after 0.00 mins
2020-06-14 17:03:05.307109 done after 0.00 mins
2020-06-14 17:03:05.309104 done after 0.

In [7]:
df

[[{'profit': 0.5,
   'loss': 0.5,
   'mean': 1.6028144981349177,
   'std': 0.9714557272660251,
   'sharpe': 1.6499099785491307}],
 [{'profit': 0.5,
   'loss': 1.0,
   'mean': 1.5902747000705064,
   'std': 0.7402265548935312,
   'sharpe': 2.1483621325895825}],
 [{'profit': 0.5,
   'loss': 1.5,
   'mean': 1.6029544855266493,
   'std': 0.7766397761897389,
   'sharpe': 2.0639613559208634}],
 [{'profit': 0.5,
   'loss': 2.0,
   'mean': 1.6475402807479136,
   'std': 0.780588607701145,
   'sharpe': 2.1106383881260644}],
 [{'profit': 0.5,
   'loss': 2.5,
   'mean': 1.710282147793881,
   'std': 0.8026179288553515,
   'sharpe': 2.1308795708476}],
 [{'profit': 0.5,
   'loss': 3.0,
   'mean': 1.5954450746996849,
   'std': 0.6605997227141475,
   'sharpe': 2.4151464492667682}],
 [{'profit': 0.5,
   'loss': 3.5,
   'mean': 1.7932089220576837,
   'std': 0.7439701043069,
   'sharpe': 2.410323898334973}],
 [{'profit': 0.5,
   'loss': 4.0,
   'mean': 1.6229287204745415,
   'std': 0.6628292340710372,
   '

In [8]:
len(df)

10000